In [26]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import nltk
import sklearn
import string
from nltk import pos_tag 
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import re
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from nltk.stem.isri import ISRIStemmer
from tensorflow.python.keras.preprocessing.text import Tokenizer 
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense , Embedding , LSTM ,GRU
from keras.layers.embeddings import Embedding

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CLS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\CLS\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\CLS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\CLS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv("Restaurant_Reviews.tsv", sep = '\t')

In [3]:
df.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


# removing negative words from English stop words

In [4]:
imp = ['against','not','don',"don't","aren't","couldn't","didn't","doesn't","hadn't","hasn't","haven't","isn't","mightn't","needn't","wasn't","weren't","wouldn't"]
stop_words = [w for w in stopwords.words('english') if w not in imp]

# text pre-processing

In [5]:
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stop_words
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)



In [6]:
df["Review"]= df["Review"].apply(lambda x : clean_text(x))

In [7]:
df["Review"].values

array(['wow love place', 'crust not good', 'not tasty texture nasty',
       'stop late may bank holiday rick steve recommendation love',
       'selection menu great price', 'get angry want damn pho',
       "honeslty didn't taste fresh",
       'potato like rubber could tell make ahead time keep warmer',
       'fry great', 'great touch', 'service prompt', 'would not go back',
       'cashier care ever say still end wayyy overprice',
       'tried cape cod ravoli chicken cranberry...mmmm',
       'disgust pretty sure human hair', 'shocked sign indicate cash',
       'highly recommended', 'waitress little slow service',
       'place not worth time let alone vegas', 'not like',
       'burrittos blah', 'food amazing', 'service also cute',
       'could care less interior beautiful', 'perform',
       "that's right....the red velvet cake.....ohhh stuff good",
       'never bring salad ask',
       'hole wall great mexican street taco friendly staff',
       'take hour get food table re

# 

In [8]:
vect = CountVectorizer(ngram_range=(1,2)).fit(df["Review"])
train_df_vectorized = vect.transform(df["Review"])

In [9]:
clfrNB = MultinomialNB(alpha = 0.1)
clfrNB.fit(train_df_vectorized, df.Liked)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [10]:
from sklearn.model_selection import cross_val_score
cv = cross_val_score(clfrNB,train_df_vectorized,df.Liked,cv=10)

In [11]:
cv

array([0.82, 0.81, 0.77, 0.78, 0.82, 0.81, 0.77, 0.8 , 0.75, 0.81])

In [12]:
cv.mean()

0.7939999999999999

In [14]:
X_train = train_df_vectorized.toarray()
X_train.shape

(1000, 5937)

In [15]:
# X_train = df["Review"].values
# y_train = df['Liked'].values

In [16]:
vect = TfidfVectorizer(ngram_range=(1,2),min_df=2,max_df=0.5).fit(df["Review"])
train_df_vectorized = vect.transform(df["Review"])

In [17]:
clfrNB = MultinomialNB(alpha = 0.1)
clfrNB.fit(train_df_vectorized, df.Liked)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [18]:
cv = cross_val_score(clfrNB,train_df_vectorized,df.Liked,cv=10)

In [19]:
cv

array([0.83, 0.82, 0.76, 0.81, 0.81, 0.84, 0.82, 0.83, 0.77, 0.8 ])

In [20]:
cv.mean()

0.8089999999999999

In [21]:
clfrNB.predict(vect.transform(['impressive']))

array([0], dtype=int64)

In [22]:
clfrRf = RandomForestClassifier(n_estimators=100,random_state=1)
clfrRf.fit(train_df_vectorized, df.Liked)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

In [23]:
cv = cross_val_score(clfrRf,train_df_vectorized,df.Liked,cv=10)

In [24]:
cv

array([0.73, 0.76, 0.78, 0.83, 0.8 , 0.81, 0.82, 0.8 , 0.86, 0.83])

In [25]:
cv.mean()

0.8019999999999999

In [28]:
tokenizer_obj = Tokenizer()
total_reviews = df['Review'].values
tokenizer_obj.fit_on_texts(df['Review'])
max_length = max([len(s.split()) for s in  total_reviews])
vocab_size = len(tokenizer_obj.word_index) +1 
X_train_tokens = tokenizer_obj.texts_to_sequences(df['Review'].values)
X_train_pad = pad_sequences(X_train_tokens,maxlen=max_length,padding='post')

In [29]:
EMBEDDDING_DIM = 100
model = Sequential()
model.add(Embedding(vocab_size,EMBEDDDING_DIM,input_length=max_length))
model.add(GRU(units=32,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [30]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])

In [32]:
model.fit(X_train_pad,df.Liked,batch_size=128,epochs=50,validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/50
800/800 [==============================] - 0s 299us/step - loss: 0.0799 - acc: 0.9825 - val_loss: 0.8603 - val_acc: 0.7950
Epoch 2/50
800/800 [==============================] - 0s 255us/step - loss: 0.0548 - acc: 0.9888 - val_loss: 0.9752 - val_acc: 0.7750
Epoch 3/50
800/800 [==============================] - 0s 249us/step - loss: 0.0581 - acc: 0.9888 - val_loss: 1.0045 - val_acc: 0.7700
Epoch 4/50
800/800 [==============================] - 0s 263us/step - loss: 0.0507 - acc: 0.9900 - val_loss: 0.9961 - val_acc: 0.7700
Epoch 5/50
800/800 [==============================] - 0s 263us/step - loss: 0.0489 - acc: 0.9900 - val_loss: 0.9418 - val_acc: 0.7900
Epoch 6/50
800/800 [==============================] - 0s 276us/step - loss: 0.0493 - acc: 0.9913 - val_loss: 0.9262 - val_acc: 0.7950
Epoch 7/50
800/800 [==============================] - 0s 255us/step - loss: 0.0466 - acc: 0.9875 - val_loss: 0.9704 - val_acc: 0.7800
Epoch 8/50
800/8